In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import random
import re

In [2]:
data = pd.read_csv('gbm_dataClustering.csv')
print(data.shape)
data.columns

(101766, 53)


Index(['Unnamed: 0', 'race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'medical_specialty', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'self_pay',
       'medicaid/medicare', 'coverByInsurance', 'f_diag', 's_diag', 't_diag',
       'clustering3', 'clustering4', 'clustering2', 'target'],
      dtype='object')

In [13]:
data = data.sort_values('target', ascending=True)

In [15]:
data = data.reset_index(drop=True)

In [17]:
data[data['target'] == 0]

,Unnamed: 0,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,num_lab_procedures,...,self_pay,medicaid/medicare,coverByInsurance,f_diag,s_diag,t_diag,clustering3,clustering4,clustering2,target
0,0,2,0,0,5,23,0,1,38,41,...,0,0,0,11,0,0,1,0,1,0
1,65526,2,0,8,0,2,6,4,22,50,...,1,0,0,7,7,5,1,0,1,0
2,65525,2,0,6,0,2,6,6,22,59,...,0,0,1,16,16,16,2,2,1,0
3,65524,2,1,7,0,2,6,7,22,58,...,0,1,0,13,9,16,1,0,1,0
4,65523,2,0,7,0,2,6,5,22,24,...,0,0,0,14,4,4,2,2,1,0
5,65521,2,1,6,0,2,6,3,22,38,...,1,0,0,14,17,17,1,0,1,0
6,65520,2,1,8,0,2,6,14,18,76,...,0,0,1,14,9,14,2,2,1,0
7,65519,2,1,8,0,2,6,3,22,65,...,0,1,0,9,6,4,1,0,1,0
8,65517,0,0,8,0,2,6,9,18,43,...,0,1,0,4,11,11,1,0,1,0
9,65515,2,0,6,0,2,6,11,22,56,...,0,1,0,11,11,9,2,2,1,0


In [19]:
unadmitted = data[:90408]

In [22]:
unadmitted

,Unnamed: 0,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,num_lab_procedures,...,self_pay,medicaid/medicare,coverByInsurance,f_diag,s_diag,t_diag,clustering3,clustering4,clustering2,target
0,0,2,0,0,5,23,0,1,38,41,...,0,0,0,11,0,0,1,0,1,0
1,65526,2,0,8,0,2,6,4,22,50,...,1,0,0,7,7,5,1,0,1,0
2,65525,2,0,6,0,2,6,6,22,59,...,0,0,1,16,16,16,2,2,1,0
3,65524,2,1,7,0,2,6,7,22,58,...,0,1,0,13,9,16,1,0,1,0
4,65523,2,0,7,0,2,6,5,22,24,...,0,0,0,14,4,4,2,2,1,0
5,65521,2,1,6,0,2,6,3,22,38,...,1,0,0,14,17,17,1,0,1,0
6,65520,2,1,8,0,2,6,14,18,76,...,0,0,1,14,9,14,2,2,1,0
7,65519,2,1,8,0,2,6,3,22,65,...,0,1,0,9,6,4,1,0,1,0
8,65517,0,0,8,0,2,6,9,18,43,...,0,1,0,4,11,11,1,0,1,0
9,65515,2,0,6,0,2,6,11,22,56,...,0,1,0,11,11,9,2,2,1,0


In [21]:
admitted = data[90409:]

In [24]:
holdout1 = unadmitted.sample(frac = 0.15)

In [53]:
data1 = unadmitted.drop(holdout1.index)

In [66]:
data1.shape

(76847, 53)

In [26]:
unadmitted.shape

(90408, 53)

In [25]:
holdout1.shape

(13561, 53)

In [27]:
holdout2 = admitted.sample(frac = 0.15)

In [68]:
admitted.shape

(11357, 53)

In [54]:
data2 = admitted.drop(holdout2.index)

In [67]:
data2.shape

(9653, 53)

In [30]:
holdout2.shape

(1704, 53)

In [37]:
holdout = holdout1.append(holdout2, ignore_index = True)

In [38]:
holdout = holdout.sample(frac=1).reset_index(drop=True)

In [55]:
new_data = data1.append(data2, ignore_index = True)

In [58]:
new_data = new_data.sample(frac=1).reset_index(drop=True)

In [60]:
holdout = holdout.drop('Unnamed: 0', axis = 1)

In [61]:
new_data = new_data.drop('Unnamed: 0', axis = 1)

In [62]:
holdout.to_csv('holdout.csv')

In [63]:
new_data.to_csv('new_data.csv')

In [64]:
holdout.shape

(15265, 52)

In [65]:
new_data.shape

(86500, 52)